In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

df_macro = pd.read_csv('macro_data_interpolated.csv')
df_macro.rename(columns = {'Unnamed: 0':'date'}, inplace=True)
df_macro.set_index(pd.to_datetime(df_macro['date']), inplace = True)
del df_macro['date']
df_shares = pd.read_csv('NumShares_Buffet.csv')
df_shares.rename(columns = {'Unnamed: 0':'date'}, inplace=True)
df_shares.set_index(pd.to_datetime(df_shares['date']), inplace = True)
del df_shares['date']

stocks_use = ['WFC', 'KO', 'AXP', 'IBM', 'WMT', 'PG', 'XOM', 'USB', 'DVA', 'GS', 'MCO', 'GHC', 'GM', 'BK', 'COP', 'PSX', 'NOV', 'MTB', 'VRSN', 'VZ', 'COST', 'DE', 'V', 'GL', 'LBTYA', 'MA', 'LBTYK', 'GE', 'SNY', 'VRSK', 'JNJ', 'MDLZ', 'UPS', 'LEE', 'LBRDA', 'QSR', 'FWONK', 'DNOW', 'KHC', 'AAPL', 'AXTA', 'KMI', 'LILA', 'LILAK', 'CHTR', 'LUV', 'DAL', 'AAL', 'UAL', 'SIRI', 'BAC', 'SYF', 'TEVA', 'STOR', 'JPM', 'PNC', 'AMZN', 'TRV', 'KR', 'OXY', 'BIIB', 'RH', 'CVX', 'ABBV', 'BMY', 'MRK', 'AON', 'TMUS', 'MMC', 'ATVI', 'HPQ', 'C', 'PARA', 'CE', 'MCK', 'MKL', 'ALLY', 'FND', 'FWONA', 'T', 'ORCL', 'PFE', 'TSM', 'LPX', 'JEF']
def get_buffet(ticker, data):
    pf = df_shares[df_shares['Sym'] == ticker]
    pf['Amount (000)'].loc['2016-12-01'] = (pf['Amount (000)'].loc['2016-09-01'].copy() + pf['Amount (000)'].loc['2017-03-01'])/2 # 오타 수정
    pf['Diff'] = pf['Amount (000)'] - pf['Amount (000)'].shift(1) # 이전꺼와의 차이
    return pf['Diff']


In [98]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(sp_url, header=0)[0]
notsp = []
stocks_saved = []
for i in stocks_use:
    if i in sp500['Symbol'].values:
        stocks_saved.append(i)
    else:
        notsp.append(i)
sp500 = sp500.set_index('Symbol')

In [100]:
sec_dict = {'NOV':'Materials','MKL':'Materials','GHC':'Communication Services','SIRI':'Communication Services',
            'LBTYA':'Real Estate','LBTYK':'Real Estate','RH':'Consumer Discretionary','FWONA':'Utilities',
            'TSM':'Information Technology','LPX':'Industrials'}

In [101]:
import requests

url = 'https://www.sec.gov/files/company_tickers_exchange.json'
headers = {'User-Agent': 'Mozilla'}
res = requests.get(url, headers=headers)
cik_list = res.json()
company_info = cik_list['data']
tickers = dict()
for p in company_info:
    if p[2] in stocks_use:
        tickers[p[2]] = [p[1],p[3]] # 이름, 거래소

regression_results = []
for s in stocks_use:
    df_fs = pd.read_csv(f'./ratio_db/{s}_ratios.csv')
    df_fs.set_index(pd.to_datetime(df_fs['date']), inplace = True)
    del df_fs['date']
    
    y_temp = get_buffet(s, df_shares)
    X = df_macro.join(df_fs).interpolate()
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.interpolate()
    #scaler = StandardScaler()
    #scaler.fit_transform(X)
    data = X.join(y_temp).fillna(method = 'ffill')
    Y = data.iloc[:,-1]
    Y = Y.replace([np.inf, -np.inf], np.nan)
    Y = Y.interpolate()
    F_Y = Y.shift(-1).dropna() # 미래를 대상으로 fitting
    #F_Y_use = F_Y.iloc[:-1]
    X_use = X.loc['2014-02-01':'2022-11-01']
    X_test = X.loc['2022-12-01':]
    
    try:
        reg = LinearRegression().fit(X_use, F_Y)
        y_pred = reg.predict(X_test)
        if s in stocks_saved:
            regression_results.append([s, int(y_pred[0]), sp500.loc[s]['Security'], tickers[s][1], sp500.loc[s]['GICS Sector']]) # ticker, 예상치, 이름, 거래소, 산업분류
            print(f'Completed Prediction for {s}: {y_pred[0]}')
        else:
            regression_results.append([s, int(y_pred[0]), tickers[s][0], tickers[s][1], sec_dict[s]])
            print(f'Completed Prediction for {s}: {y_pred[0]}')
    except:
        print(f'Failed to Predict {s}')

    

Completed Prediction for WFC: -30245.660028225742
Completed Prediction for KO: -2.1852092531044036
Completed Prediction for AXP: 135.8381019572189
Completed Prediction for IBM: -1440.9701313734986
Completed Prediction for WMT: -4111.0682489369065
Completed Prediction for PG: -406.00813763123006
Completed Prediction for XOM: -13153.565518570482
Completed Prediction for USB: -49992.66009289562
Completed Prediction for DVA: 394.70142387758824
Completed Prediction for GS: 82.29837634929572
Completed Prediction for MCO: -212.12533430159783
Completed Prediction for GHC: -212.19126277020405
Completed Prediction for GM: 1419.2175972698606
Completed Prediction for BK: -5665.4482005722675
Completed Prediction for COP: 573.2608107994456
Completed Prediction for PSX: -6970.463956391672
Completed Prediction for NOV: -200.7939292620431
Completed Prediction for MTB: 466.3174078471784
Completed Prediction for VRSN: 159.17784290180657
Completed Prediction for VZ: 12865.955133029725
Completed Prediction

In [102]:
regression_results.sort(key = lambda x:x[1], reverse=True)

In [103]:
len(regression_results)

67

In [105]:
print(regression_results)

[['TSM', 78301, 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD', 'NYSE', 'Information Technology'], ['AAPL', 45850, 'Apple Inc.', 'Nasdaq', 'Information Technology'], ['OXY', 36550, 'Occidental Petroleum', 'NYSE', 'Energy'], ['VZ', 12865, 'Verizon', 'NYSE', 'Communication Services'], ['PARA', 11039, 'Paramount Global', 'Nasdaq', 'Communication Services'], ['LPX', 7700, 'LOUISIANA-PACIFIC CORP', 'NYSE', 'Industrials'], ['MRK', 6803, 'Merck & Co.', 'NYSE', 'Health Care'], ['AAL', 5309, 'American Airlines Group', 'Nasdaq', 'Industrials'], ['SIRI', 4072, 'SIRIUS XM HOLDINGS INC.', 'Nasdaq', 'Communication Services'], ['BMY', 2324, 'Bristol Myers Squibb', 'NYSE', 'Health Care'], ['KMI', 1833, 'Kinder Morgan', 'NYSE', 'Energy'], ['GM', 1419, 'General Motors', 'NYSE', 'Consumer Discretionary'], ['ABBV', 1140, 'AbbVie', 'NYSE', 'Health Care'], ['PFE', 941, 'Pfizer', 'NYSE', 'Health Care'], ['COST', 623, 'Costco', 'Nasdaq', 'Consumer Staples'], ['COP', 573, 'ConocoPhillips', 'NYSE', 'Energy'], ['RH

for i in X_use.isna().sum():
    if i != 0:
        print(i)

max = 0
for idx, val in enumerate(X_use.max()):
    if val >= max:
        print(val, idx)
        max = val